In [57]:
f = 'j2 j1 0e 0 1 0'

re.sub(' ', '  ', f)
re.sub(r'\n[01]\n', '  ', f)

'j2 j1 0e 0 1 0'

In [76]:
import os
import re

path1 = r"C:\Users\giaco\Desktop\240911_20875-Software-Engineering\HW01\outputs"
path2 = r"C:\Users\giaco\Downloads\outputs"
for file in os.listdir(path1):
    print(file)
    # JACK  
    f1 = open(path1 + "/" + file, 'r').read()
    f1 = f1.replace(' ', '').replace('\n\n', '\n')
    f1 = f1.splitlines()[1:]
    f1 = '\n'.join(f1)
    f1 = re.sub(r'[^01]', '', f1)

    # VEZZO
    f2 = open(path2 + '/' + file[:-4] + '_output.txt', 'r', encoding='utf-16').read()
    f2 = re.sub(r'---.+---', '', f2)
    f2 = f2.splitlines()[1:]
    f2 = '\n'.join(f2)
    f2 = f2.replace(' ', '')
    f2 = re.sub(r'[^01]', '', f2)

    for i in range(len(f2)):
        if f1[i] != f2[i]:
            print(f1[i], f2[i])
            print(i)


ag24_00.txt
ag24_01.txt
ag24_02.txt
ag24_03.txt
ag24_04.txt
ag24_05.txt
ag24_06.txt
ag24_07.txt
ag24_08.txt
ag24_09.txt
ag24_10.txt
ag24_11.txt
ag24_12.txt
ag24_13.txt
ag24_14.txt
ag24_15.txt
